#### Step 1: Set Up Environment and Import Libraries

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

#### Step 2: Download and Prepare the Dataset

In [2]:
import os
os.getcwd()

'C:\\Users\\saura\\Neural Network - EXL'

In [3]:
data_dir = 'image'

# Define transformations for the training and validation sets
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Create datasets
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'seg_train'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'seg_test'), transform=transform)

In [5]:
# Split train dataset into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [6]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#### Step 3: Define the DCNN Model

In [7]:
class SimpleDCNN(nn.Module):
    def __init__(self, num_classes=6):
        super(SimpleDCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        
        x = x.view(-1, 128 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

model = SimpleDCNN(num_classes=6)

#### Define Loss Function and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Train the Model

In [9]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}')

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
    
    val_accuracy = val_correct.double() / len(val_dataset)
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {val_accuracy:.4f}')

Epoch 1/5, Loss: 0.9737
Validation Loss: 0.7270, Accuracy: 0.7282
Epoch 2/5, Loss: 0.6429
Validation Loss: 0.5975, Accuracy: 0.7766
Epoch 3/5, Loss: 0.5006
Validation Loss: 0.5182, Accuracy: 0.8151
Epoch 4/5, Loss: 0.3787
Validation Loss: 0.4998, Accuracy: 0.8208
Epoch 5/5, Loss: 0.2874
Validation Loss: 0.5687, Accuracy: 0.8276


#### Evaluate the Model

In [10]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

Accuracy: 0.8227
              precision    recall  f1-score   support

   buildings       0.81      0.74      0.77       437
      forest       0.97      0.90      0.94       474
     glacier       0.71      0.88      0.79       553
    mountain       0.83      0.72      0.77       525
         sea       0.90      0.79      0.84       510
      street       0.78      0.90      0.84       501

    accuracy                           0.82      3000
   macro avg       0.83      0.82      0.82      3000
weighted avg       0.83      0.82      0.82      3000

